In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
from syft.core.node.common.action.get_object_action import GetObjectAction
from syft.core.node.common.action.function_or_constructor_action import RunFunctionOrConstructorAction
from syft.core.node.common.action.common import Action
from syft.proto.core.node.common.plan.plan_pb2 import Plan as Plan_PB

from typing import List
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.common.serde.deserialize import _deserialize
from syft.core.common.object import Serializable
from syft.proto.core.node.common.action.action_pb2 import Action as Action_PB
from syft.core.node.common.plan.plan import Plan
import torch as th
import sys

# Plans

## Serialization

In [3]:
a1 = GetObjectAction(id_at_location=UID(), address=Address(), reply_to=Address(), msg_id=UID())
a2 = RunFunctionOrConstructorAction(path="torch.Tensor.add",args=tuple(),kwargs={},id_at_location=UID(),
                                    address=Address(),msg_id=UID())

In [4]:
plan = Plan([a1,a2])

In [5]:
blob = plan.serialize()
plan_reconstructed = sy.deserialize(blob=blob)

In [6]:
# %debug

In [7]:
assert isinstance(plan_reconstructed, Plan)
assert all(isinstance(a, Action) for a in plan_reconstructed.actions)

## Execution

In [8]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [9]:
t = th.tensor([1,2,3])
tensor_pointer = t.send(alice_client)

In [10]:
action = GetObjectAction(id_at_location=tensor_pointer.id_at_location, address=Address(),
                         reply_to=Address(), msg_id=UID())
plan = Plan([action])

In [11]:
plan_pointer = plan.send(alice_client)
result = plan_pointer.execute()

In [12]:
x = result.get()

In [13]:
x

In [14]:
result

In [15]:
plan_pointer

In [16]:
alice_client

<VirtualMachineClient: alice Client>